In [ ]:
import utils
import skimage
import numpy as np
import pathlib
from skimage import io


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\adria\miniconda3\envs\tdt4195\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\adria\miniconda3\envs\tdt4195\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\adria\miniconda3\envs\tdt4195\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  Fi

In [17]:
def extract_boundary(im: np.ndarray) -> np.ndarray:
    """
        A function that extracts the inner boundary from a boolean image.

        args:
            im: np.ndarray of shape (H, W) with boolean values (dtype=np.bool)
        return:
            (np.ndarray) of shape (H, W). dtype=np.bool
    """
    structuring_element = np.array([
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]
    ], dtype=bool)
    boundary = im.astype(np.int32)
    boundary = boundary - skimage.morphology.erosion(boundary,structuring_element)
    boundary = boundary.astype(bool)
    return boundary

In [18]:
im = utils.read_im("images/blood-vessels.png")
binary_image = (im != 0)
boundary = extract_boundary(binary_image)

assert im.shape == boundary.shape, "Expected image shape ({}) to be same as resulting image shape ({})".format(
        im.shape, boundary.shape)
assert boundary.dtype == bool, "Expected resulting image dtype to be bool. Was: {}".format(
        boundary.dtype)

boundary = utils.to_uint8(boundary)
utils.save_im("blood-vessels-boundary.png", boundary)

Saving image to: image_processed\blood-vessels-boundary.png


In [19]:
def remove_noise(im: np.ndarray) -> np.ndarray:
    """
        A function that removes noise in the input image.
        args:
            im: np.ndarray of shape (H, W) with boolean values (dtype=bool)
        return:
            (np.ndarray) of shape (H, W). dtype=bool
    """
    im = skimage.morphology.erosion(im,skimage.morphology.disk(7))
    im = skimage.morphology.dilation(im,skimage.morphology.disk(13))
    return im

In [20]:
im = utils.read_im("images/noisy.png")

binary_image = (im != 0)
noise_free_image = remove_noise(binary_image)

assert im.shape == noise_free_image.shape, "Expected image shape ({}) to be same as resulting image shape ({})".format(
        im.shape, noise_free_image.shape)
assert noise_free_image.dtype == bool, "Expected resulting image dtype to be bool. Was: {}".format(
        noise_free_image.dtype)

noise_free_image = utils.to_uint8(noise_free_image)
utils.save_im("noisy-filtered.png", noise_free_image)

Saving image to: image_processed\noisy-filtered.png


In [21]:
def distance_transform(im: np.ndarray) -> np.ndarray:
    """
        A function that computes the distance to the closest boundary pixel.

        args:
            im: np.ndarray of shape (H, W) with boolean values (dtype=np.bool)
        return:
            (np.ndarray) of shape (H, W). dtype=np.int32
    """
    assert im.dtype == bool
    structuring_element = np.array([
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]
    ], dtype=bool)
    temp_image = im.astype(np.int32)
    erosion_count = 0
    h,w = temp_image.shape
    result = np.zeros_like(temp_image)

    while np.any(temp_image != 0):
        temp_err = skimage.morphology.erosion(temp_image,structuring_element)
        erosion_count += 1
        temp_border = temp_image - temp_err
        for i in range(h):
            for j in range(w):
                if temp_border[i,j] == 1:
                    result[i,j] += erosion_count
        temp_image = temp_err

    return result

In [22]:
im = utils.read_image("noisy-filtered.png", image_folder=pathlib.Path("image_processed"))
binary_image = (im != 0)
distance = distance_transform(binary_image)

assert im.shape == distance.shape, "Expected image shape ({}) to be same as resulting image shape ({})".format(
        im.shape, distance.shape)
assert distance.dtype == np.int32, "Expected resulting image dtype to be np.int32. Was: {}".format(
        distance.dtype)

distance = utils.to_uint8(distance)
utils.save_im("noisy-distance.png", distance)

Reading image: image_processed\noisy-filtered.png
Saving image to: image_processed\noisy-distance.png


c:\Users\adria\OneDrive - NTNU\GitHub\image-processing\Image-Prosessing\utils.py:28: UserWarning: Image min/max is outside the range [0.0, 1.0]. Squashing the image to this range. (Can be safely ignored)
  warnings.warn("Image min/max is outside the range [0.0, 1.0]. Squashing the image to this range. (Can be safely ignored)")


In [23]:
def fill_holes(im: np.ndarray, starting_points: list, num_iterations: int) -> np.ndarray:
    """
        A function that takes a binary image (im),  and a set of points 
        indicating position of holes, and fills the holes.

        args:
            im: np.ndarray of shape (H, W) with boolean values (dtype=np.bool)
            starting_points: list of list containing starting points (row, col). Ex:
                [[row1, col1], [row2, col2], ...]
            num_iterations: integer defining the number of iterations to apply the 
                            hole filling algorithm
        return:
            (np.ndarray) of shape (H, W). dtype=np.bool
    """
    structuring_element = np.array([
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]
    ], dtype=bool)

    x = np.zeros_like(im)
    h,w = im.shape
    for row,col in starting_points:
        x[row,col] = 1

    im_compliment = np.zeros_like(im)
    for i in range(h):
        for j in range(w):
            if im[i,j] == 0:
                im_compliment[i,j] = 1

    for i in range(num_iterations):
        x_dil = skimage.morphology.dilation(x,structuring_element)
        x = np.zeros_like(im)
        for i in range(h):
            for j in range(w):
                if x_dil[i,j] == 1 and im_compliment[i,j] == 1:
                    x[i,j] = 1
    
    result = np.zeros_like(im)

    for i in range(h):
        for j in range(w):
            if x[i,j] == 1 or im[i,j] == 1:
                result[i,j] = 1

    return result

In [24]:
im = io.imread('images/cards.png', as_gray=True)
binary_image = im != 0
starting_points = [ 
    # (row, column)
    [50, 80],
    [275, 80],
    [50, 175],
    [275, 175],
    [50,390],
    [275, 390],
    [175,650]
]
num_iterations = 50

result = fill_holes(binary_image, starting_points, num_iterations)

assert im.shape == result.shape, "Expected image shape ({}) to be same as resulting image shape ({})".format(
        im.shape, result.shape)
assert result.dtype == bool, "Expected resulting image dtype to be bool. Was: {}".format(
        result.dtype)

result = utils.to_uint8(result)
utils.save_im("cards-filled.png", result)

Saving image to: image_processed\cards-filled.png


In [25]:
def remove_noise(im: np.ndarray) -> np.ndarray:
    """
        A function that removes noise in the input image.
        args:
            im: np.ndarray of shape (H, W) with boolean values (dtype=bool)
        return:
            (np.ndarray) of shape (H, W). dtype=bool
    """
    im = skimage.morphology.erosion(im,skimage.morphology.disk(7))
    im = skimage.morphology.dilation(im,skimage.morphology.disk(13))
    return im

In [26]:
im = utils.read_im("images/noisy.png")

binary_image = (im != 0)
noise_free_image = remove_noise(binary_image)

assert im.shape == noise_free_image.shape, "Expected image shape ({}) to be same as resulting image shape ({})".format(
        im.shape, noise_free_image.shape)
assert noise_free_image.dtype == bool, "Expected resulting image dtype to be bool. Was: {}".format(
        noise_free_image.dtype)

noise_free_image = utils.to_uint8(noise_free_image)
utils.save_im("noisy-filtered.png", noise_free_image)

Saving image to: image_processed\noisy-filtered.png


In [27]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a threshold value

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    
    hist , bins = np.histogram(im,bins = 256, range=(0,256))
    p = hist / np.sum(hist)

    k_list = np.arange(0,len(hist))
    k_max = 0
    sigma_b_max = 0
    global_mean = np.sum(bins[0:-1] * p)

    for k in k_list:
        cum_sum = np.sum(p[:k+1])
        cum_mean = np.sum(bins[0:k+1] * p[0:k+1])
        if cum_sum == 1 or cum_sum == 0:
            sigma_b = 0
        else:
            sigma_b = (global_mean * cum_sum - cum_mean)**2 / (cum_sum * (1-cum_sum))
        if sigma_b > sigma_b_max:
            sigma_b_max = sigma_b
            k_max = k

    threshold = k_max
    
    return threshold

In [28]:
impaths_to_segment = [
    pathlib.Path("thumbprint.png"),
    pathlib.Path("rice-shaded.png")
]
for impath in impaths_to_segment:
    im = utils.read_image(impath)
    threshold = otsu_thresholding(im)
    print("Found optimal threshold:", threshold)

    # Segment the image by threshold
    segmented_image = (im >= threshold)
    assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
            im.shape, segmented_image.shape)
    assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
            segmented_image.dtype)

    segmented_image = utils.to_uint8(segmented_image)

    save_path = "{}-segmented.png".format(impath.stem)
    utils.save_im(save_path, segmented_image)

Reading image: images\thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed\thumbprint-segmented.png
Reading image: images\rice-shaded.png
Found optimal threshold: 134
Saving image to: image_processed\rice-shaded-segmented.png


In [1]:
def region_growing(im: np.ndarray, seed_points: list, T: int) -> np.ndarray:
    """
        A region growing algorithm that segments an image into 1 or 0 (True or False).
        Finds candidate pixels with a Moore-neighborhood (8-connectedness). 
        Uses pixel intensity thresholding with the threshold T as the homogeneity criteria.
        The function takes in a grayscale image and outputs a boolean image

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
            seed_points: list of list containing seed points (row, col). Ex:
                [[row1, col1], [row2, col2], ...]
            T: integer value defining the threshold to used for the homogeneity criteria.
        return:
            (np.ndarray) of shape (H, W). dtype=bool
    """
    segmented = np.zeros_like(im).astype(bool)
    im = im.astype(float)
    for row, col in seed_points:
        segmented[row, col] = True

    h,w = im.shape

    seed_queue = []
    for i in seed_points:
        seed_queue.append(i)
    
    while seed_queue:
        seed = seed_queue.pop(0)
        x_seed , y_seed = seed
        queue = [seed]
        while queue:
            poi = queue.pop(0)
            x , y = poi

            neighbors = []

            for i in [-1,0,1]:
                for j in [-1,0,1]:
                    if i == 0 and j == 0:
                        continue
                    else:
                        neighbors.append([x+i,y+j])
            
            for neighbor in neighbors:
                x,y = neighbor
                if x < 0 or y < 0 or x >= h or y >= w:
                    continue
                else:
                    if segmented[x,y] == True:
                        continue
                    else:
                        if np.abs(im[x,y] - im[x_seed,y_seed]) <= T:
                            queue.append([x,y])
                            segmented[x,y] = True

    return segmented

NameError: name 'np' is not defined

In [30]:
im = utils.read_image("defective-weld.png")

seed_points = [ # (row, column)
    [254, 138], # Seed point 1
    [253, 296], # Seed point 2
    [233, 436], # Seed point 3
    [232, 417], # Seed point 4
]
intensity_threshold = 50
segmented_image = region_growing(im, seed_points, intensity_threshold)

assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
    im.shape, segmented_image.shape)
assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
        segmented_image.dtype)

segmented_image = utils.to_uint8(segmented_image)
utils.save_im("defective-weld-segmented.png", segmented_image)

Reading image: images\defective-weld.png
Saving image to: image_processed\defective-weld-segmented.png


c:\Users\adria\miniconda3\envs\tdt4195\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: image_processed\defective-weld-segmented.png is a low contrast image
  return func(*args, **kwargs)
